In [1]:
import io # To be able to read strings as files
import string # To extract a list of lowercase letters
import numpy as np
import pandas as pd

# Import Biopython for file I/O.
import Bio.SeqIO
from Bio import AlignIO
import Bio.Phylo

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
# Read alignment with all species

def compute_similarity(seq_1, seq_2):
    # Make sure they are the same length. 
    if len(seq_1) != len(seq_2):
        raise ValueError('Sequences must be the same length!')
        
    # Make both sequences lowercase.
    seq_1 = seq_1.lower()
    seq_2 = seq_2.lower()
        
    # Set up counters of length and similarity.
    comp_length = 0
    num_sim = 0
    
    # Iterate through each position in the sequences.
    for base in range(len(seq_1)):
        
        # Ensure we are not comparing gaps.
        if (seq_1[base] != '-') and (seq_2[base] != '-'):
            
            # Increase the counter for compared length.
            comp_length += 1
            
            # Compare the two positions.
            if seq_1[base] == seq_2[base]:
                
                # Increase the similarity counter.
                num_sim += 1
                
    # Compute and return the percent similarity.
    score = num_sim  / comp_length
    return score


### The Neighbor-Joining algorithm
##### The procedure begins by assuming that the tree's structure has no hierarchy (the starlike tree). The technique iteratively locates the two nearest nodes and combines them to create a new parental node using the pairwise distance between the nodes as input (like node u in the second schematic). This operation is repeated, adjusting the distance measure as fewer nodes are mismatched. Once two nodes are connected to one another, the algorithm no longer considers them. These nodes can only be linked to the remainder of the tree through the newly generated parental node.

In [2]:
def s_matrix(mat):
    # Get the number of entries in matrix
    n = len(mat)
    
    # Compute the starlike length
    s_star = 1 / (n - 1) * np.sum(np.triu(mat))
    # Initialize matrix to save entries
    s_mat = np.ones_like(mat) * s_star

    # Loop through rows
    for i in range(n):
        # Loop through columns
        for j in range(n):
            # Define list of index different from i and j
            idx_out = np.arange(n)
            idx_out = np.delete(idx_out, [i, j])
            
            first_term = 1 / (2 * (n - 2)) * np.sum(mat[i, idx_out] + \
                                                    mat[j, idx_out])
            
            # Second term
            second_term = 1 / 2 * mat[i, j]
            
            reduced_mat = np.delete(mat, [i, j], axis=0)
            reduced_mat = np.delete(reduced_mat, [i, j], axis=1)
            third_term = 1 / (n - 2) * np.sum(np.triu(reduced_mat))
            
            # Add result to the s matrix
            s_mat[i, j] = first_term + second_term + third_term
            
    np.fill_diagonal(s_mat, 0)
    
    return s_mat

# For New File output.maf

In [3]:
aln = AlignIO.read("output_all.maf", "maf")
print(aln)

Alignment with 6 rows and 4110 columns
GTGTGTTAATCTTACAACCAGAACTCAATTACCCCCTGCATACA...TAT sars2
GTGTGTTAATCTTACAACCAGAACTCAATTACCCCCTGCATACA...GCT alpha
TTTTACAACCAGAACTCAATTACCCCCTGCATACACTAATTCTT...TAC beta
TTTACAAACAGAACTCAATTACCCTCTGCATACACTAATTCTTT...TTT gamma
CAGAACTCAATCATACACTAATTCTTTCACACGTGGTGTTTATT...CAG omicron
CCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTG...TTG delta


In [4]:
taxa = []
for record in aln:
    taxa.append(record.id)

print(taxa)

['sars2', 'alpha', 'beta', 'gamma', 'omicron', 'delta']


### Implementation 
##### Although the method appears to involve a lot of unpleasant sums, it is actually rather simple to put into practise. The distance matrix that the method will employ has to be constructed first. In order to do this, I suggest using a function we've already created that compares two sequences pairwise and produces a square symmetric matrix of all the species that are currently accessible.

##### We will use 1 - our similarity score as a distance metric since we need something to quantify how far apart two sequences are. This makes sense since, according to the suggested measure, the distance between one species and itself at the diagonal will be zero.

In [5]:
sim_mat = np.zeros([len(aln), len(aln)])

for i, sp1 in enumerate(aln):
    # Extract species 1 sequences
    seq_1 = sp1.seq
    # loop through species in aln
    for j, sp2 in enumerate(aln):
        # Extract species 2 sequences
        seq_2 = sp2.seq
        sim_mat[i, j] = 1 - compute_similarity(seq_1, seq_2) ## distance matrix
        
print('Distance matrix for all {:d} sequences:'.format(len(aln)))
print(sim_mat)

Distance matrix for all 6 sequences:
[[0.         0.68613139 0.71776156 0.74355231 0.71995134 0.74379562]
 [0.68613139 0.         0.66545012 0.72871046 0.72408759 0.73819951]
 [0.71776156 0.66545012 0.         0.74136253 0.72092457 0.73746959]
 [0.74355231 0.72871046 0.74136253 0.         0.74452555 0.73552311]
 [0.71995134 0.72408759 0.72092457 0.74452555 0.         0.74939173]
 [0.74379562 0.73819951 0.73746959 0.73552311 0.74939173 0.        ]]


In [6]:
s_mat = s_matrix(sim_mat)
print('Total branch length difference with respect to starlike topology')
print(s_mat)

Total branch length difference with respect to starlike topology
[[0.         2.17305353 2.18381995 2.18287713 2.17542579 2.18166058]
 [2.17305353 0.         2.16624088 2.18403285 2.18607056 2.18743917]
 [2.18381995 2.16624088 0.         2.18531022 2.17944039 2.18202555]
 [2.18287713 2.18403285 2.18531022 0.         2.17740268 2.16721411]
 [2.17542579 2.18607056 2.17944039 2.17740268 0.         2.17849757]
 [2.18166058 2.18743917 2.18202555 2.16721411 2.17849757 0.        ]]


In [7]:
mat = str(s_mat).replace(' [', '').replace('[', '').replace(']', '')
with open('matrix.txt', 'w') as f:
    f.write(mat)# Save the simulation matrix

### UPGMA Implementation
##### The simplest distance-matrix approach is the UPGMA, which uses sequential clustering to create a rooted phylogenetic tree. To construct the distance matrix, all sequences were initially compared using pairwise alignment as we do above.
##### It begins by grouping the two taxa that have the least pairwise distance in the distance matrix, given the distance matrix. At the halfway point or distance between them, a node is positioned. The new cluster is then treated as a single taxon to produce a reduced matrix. To build a smaller matrix, the distances between this new composite taxon and all other taxa are determined. Another freshly reduced matrix is produced after repeating the grouping procedure. Once every taxon has been added to the tree, the iteration is complete. The final taxon introduced is regarded as the outgroup that can produce a rooted tree. The fundamental premise of the UPGMA approach is that a molecular clock is operating and that all taxa are equally far from the root and develop at a constant rate. Thus, incorrect tree topologies are frequently generated using UPGMA. However, because of its quick computation speed, it has been widely used in the clustering analysis of DNA microarray data.

In [8]:
"""
UPGMA - Unweighted pair group method using arithmetic averages
Takes as input the distance matrix of species as a numpy array
Returns tree either as a dendrogram or in Newick format
"""
from ete3 import Tree
#%% import modules
import numpy as np
import itertools

#%% class for nodes
class Node:
    """
    Data structure to store node of a UPGMA tree
    """

    def __init__(self, left=None, right=None, up_height=0.0, down_height=0.0):
        """
        Creating a node.
        For a single taxon, set taxon name as self.left, leave right as none.
        For an operational taxonomic unit(OTU) set left and right to child nodes.

        Parameters
        ----------
        left : default = none, taxon label
        right : default = none, taxon label
        up_height : float, default = 0.0, dist to parent node, if any
        down_height : float, default = 0.0, dist to child node, if any
        """
        self.left = left
        self.right = right
        self.uh = up_height
        self.dh = down_height

    def leaves(self) -> list:
        """
        Method to find the taxa under any given node, effectively equivalent to
        finding leaves of a binary tree. Only lists original taxa and not OTUs.

        Returns a list of node names, not nodes themselves.
        """
        if self == None:
            return []
        if self.right == None:
            return [self.left]
        leaves = self.left.leaves() + self.right.leaves()
        return leaves

    def __len__(self) -> int:
        """
        Method to define len() of a node.

        Returns the number of original taxa under any given node.
        """
        return sum(1 for taxa in self.leaves())

    def __repr__(self) -> str:
        """
        Method to give readable print output
        """
        return "-".join(self.leaves())


#%% class for UPGMA
class UPGMA:
    def __init__(self, dist_matrix: np.ndarray, taxa: list):
        """
        Initialize an UPGMA class.
        Takes a nxn distance matrix as input. A list of n taxon id is required
        in the same order as the distance matrix row/column

        Parameters
        ----------
        dist_matrix : numpy array, distance matrix of species
        taxa : list of int or str to identify taxa
        """
        self.distances = dist_matrix
        self.taxa = taxa
        self.build_tree(self.distances, self.taxa)

    def build_tree(self, dist_matrix: np.ndarray, taxa: list) -> Node:
        """
        Method to construct a tree from a given distance matrix and taxa list.

        Parameters
        ----------
        dist_matrix : np.ndarray of pairwise distances
        taxa : list of taxa id. Elements of lists have to be unique

        Returns the root node for constructed tree.
        """
        # creating node for each taxa
        nodes = list(Node(taxon) for taxon in taxa)

        # dictionary row/column id -> node
        rc_to_node = dict([i, j] for i, j in enumerate(nodes))
        
        # dictionary taxa -> row/column id
        taxa_to_rc = dict([i, j] for j, i in enumerate(taxa))
        
        # make copy of dist matrix to work on
        work_matrix = dist_matrix
        # set all diagonal elements to infinity for ease of finding least distance
        work_matrix = np.array(work_matrix, dtype=float)
        np.fill_diagonal(work_matrix, np.inf)

        # loop
        while len(nodes) > 1:
            # finding (row, col) of least dist
            least_id = np.unravel_index(work_matrix.argmin(), work_matrix.shape, "C")
            least_dist = work_matrix[least_id[0], least_id[1]]
            # nodes corresponding to (row, col)
            node1, node2 = rc_to_node[least_id[0]], rc_to_node[least_id[1]]
            
            # add OTU with node1 and node2 as children. set heights of nodes
            new_node = Node(node2, node1)
            nodes.append(new_node)
            node1.uh = least_dist / 2 - node1.dh
            node2.uh = least_dist / 2 - node2.dh
            new_node.dh = least_dist / 2
            nodes.remove(node1)
            nodes.remove(node2)
           
            # create new working distance matrix
            work_matrix = self.update_distance(dist_matrix, nodes, taxa_to_rc)
            
            # update row/col id -> node dictionary
            rc_to_node = dict([i, j] for i, j in enumerate(nodes))
        # set tree to root
        self.tree = nodes[0]

    def update_distance(
        self, dist_matrix: np.ndarray, nodes: list, taxa_to_rc: dict
    ) -> np.ndarray:
        """
        Method to make a new distance matrix with newer node list.

        Parameters
        ----------
        dist_matrix : np.ndarray of pairwise distances for all taxa
        nodes : list of updated nodes
        taxa_to_rc : dict for taxa -> row/col id

        Returns np.ndarray of pairwise distances for updated nodes
        """
        # dictionary for node -> row/col id
        node_to_rc = dict([i, j] for j, i in enumerate(nodes))
        
        rc = len(nodes)
        new_dist_matrix = np.zeros((rc, rc), dtype=float)
        for node1 in nodes:
            row = node_to_rc[node1]
            for node2 in nodes:
                node_pairs = list(itertools.product(node1.leaves(), node2.leaves()))
                col = node_to_rc[node2]
                new_dist_matrix[row, col] = sum(
                    dist_matrix[taxa_to_rc[i], taxa_to_rc[j]] for i, j in node_pairs
                ) / len(node_pairs)
        np.fill_diagonal(new_dist_matrix, np.inf)
        return new_dist_matrix


def tree_to_newick(t) -> str:
    """
    Function to convert tree to Newick, slightly modified form of the tree.py version.
    Takes the root node of an UPGMA tree as input
    """
    if t.right == None:
        return t.left + ":" + str(t.uh)
    else:
        return (
            "("
            + ",".join([tree_to_newick(x) for x in [t.left, t.right]])
            + "):"
            + str(t.uh)
        )

def tree_to_newick(t) -> str:
    """
    Function to convert tree to Newick, slightly modified form of the tree.py version.
    Takes the root node of an UPGMA tree as input
    """
    if t.right == None:
        return t.left + ":" + str(t.uh)
    else:
        return (
            "("
            + ",".join([tree_to_newick(x) for x in [t.left, t.right]])
            + "):"
            + str(t.uh)
        )
#%% main
if __name__ == "__main__":
    # data from table 3 of Fitch and Margoliash, Construction of Phylogenetic trees
    #taxa = list(map(chr, range(97,97+len(aln))))
    x = UPGMA(sim_mat, taxa).tree
    print(tree_to_newick(x))
    t=Tree(tree_to_newick(x)+";")
    print(t)

((delta:0.3677615571776156,gamma:0.3677615571776156):0.0026763990267639204,(((beta:0.3327250608272506,alpha:0.3327250608272506):0.018248175182481785,sars2:0.3509732360097324):0.009854014598540128,omicron:0.36082725060827253):0.009610705596106994):0.0

      /-delta
   /-|
  |   \-gamma
  |
--|         /-beta
  |      /-|
  |   /-|   \-alpha
  |  |  |
   \-|   \-sars2
     |
      \-omicron
